In [1]:
import polars as pl
import quak

df = pl.read_csv("a_df.csv")
reference_frame = pl.read_csv("b_reference_frame.csv")

widget = quak.Widget(df)
widget

Widget(sql='SELECT * FROM "df"', temp_indexes=True)

In [2]:
import bpy

blend_file_path = "housing_data.blend"
bpy.ops.wm.open_mainfile(filepath=blend_file_path)

No MNSession found to load for this .blend file.


{'FINISHED'}

2024-08-30 20:00:59.502 Blender[10274:268108] +[IMKClient subclass]: chose IMKClient_Legacy
2024-08-30 20:00:59.502 Blender[10274:268108] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Read blend: "/Users/jan-hendrik/projects/ipyblender-experimental/california_housing/housing_data.blend"
Info: Saved "housing_data.blend"


In [3]:
# Load to Blender
import bpy

vertices = [(row["longitude_normalized"], row["latitude_normalized"], 0) for row in reference_frame.iter_rows(named=True)]

# Create a new mesh and object
mesh = bpy.data.meshes.new(name="NormalizedMesh")
obj = bpy.data.objects.new(name="CaliforninaNormalizedObject", object_data=mesh)

# Link the object to the current collection
bpy.context.collection.objects.link(obj)

# Create the mesh from the vertices, edges, and faces
mesh.from_pydata(vertices, [], [])

# Update mesh with new data
mesh.update()

# Add the Geometry Nodes modifier and assign the node group
modifier = obj.modifiers.new(name="GeometryNodes", type='NODES')
modifier.node_group = bpy.data.node_groups["geo_house"]

# Optionally, set the object as active and select it
bpy.context.view_layer.objects.active = obj
obj.select_set(True)

reference_frame.head()

short_id,longitude_normalized,latitude_normalized
str,f64,f64
"""#aaa""",3.416335,3.03932
"""#aab""",1.922311,5.377258
"""#aac""",4.561753,2.019129
"""#aad""",2.440239,5.217853
"""#aae""",6.513944,1.370882


In [7]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np

output = widgets.Output()

def on_data_change(change):
    output.clear_output(wait=True)
    with output:


        widget_df = widget.data().pl()

        # Step 1: Create plotting_frame with the same short_id as reference_frame and initialize custom_plotting to zero
        plotting_frame = reference_frame.with_columns([pl.lit(0).alias("custom_plotting")])

        # Step 2: Update custom_plotting with the value of median_house_value where short_id matches
        # First, we need to join the widget_df with plotting_frame based on short_id
        plotting_frame = plotting_frame.join(widget_df.select(["short_id", "median_house_value"]), on="short_id", how="left")

        # Step 3: Replace the zero values in custom_plotting with median_house_value where there is a match
        plotting_frame = plotting_frame.with_columns(
            pl.when(pl.col("median_house_value").is_not_null())
            .then(pl.col("median_house_value"))
            .otherwise(pl.col("custom_plotting"))
            .alias("custom_plotting")
        ).select(["short_id", "custom_plotting"])


        custom_plotting_list = plotting_frame["custom_plotting"].to_list()
        normalized_custom_plotting_list = list(np.interp(custom_plotting_list, 
                                                        (min(custom_plotting_list), max(custom_plotting_list)), 
                                                        (0.1, 3)))

        obj = bpy.data.objects['CaliforninaNormalizedObject']
        attr_name = 'median_house_value'

        # Create or get the attribute
        attr = obj.data.attributes.get(attr_name)
        if attr is None:
            attr = obj.data.attributes.new(
                name=attr_name,
                type='FLOAT',
                domain='POINT'
            )


        attr.data.foreach_set('value', normalized_custom_plotting_list)
        obj.data.update()

        print(len(normalized_custom_plotting_list))
        normalized_custom_plotting_list[:3]
        
        print(widget.data().df())

widget.observe(on_data_change, names= ["sql"])
display(output)

Output()